### --- Day 16: Proboscidea Volcanium ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/16

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 16);
WriteLines(inputLines);

Loading puzzle file: Day16.txt
Total lines: 51
Max line length: 67

Valve OS has flow rate=0; tunnels lead to valves EE, CL
Valve EN has flow rate=0; tunnels lead to valves CL, GV
Valve RR has flow rate=24; tunnels lead to valves FS, YP
Valve VB has flow rate=20; tunnels lead to valves UU, EY, SG, ZB
Valve UU has flow rate=0; tunnels lead to valves OT, VB


In [4]:
string[] testInputLines = 
[
    "Valve AA has flow rate=0; tunnels lead to valves DD, II, BB",
    "Valve BB has flow rate=13; tunnels lead to valves CC, AA",
    "Valve CC has flow rate=2; tunnels lead to valves DD, BB",
    "Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE",
    "Valve EE has flow rate=3; tunnels lead to valves FF, DD",
    "Valve FF has flow rate=0; tunnels lead to valves EE, GG",
    "Valve GG has flow rate=0; tunnels lead to valves FF, HH",
    "Valve HH has flow rate=22; tunnel leads to valve GG",
    "Valve II has flow rate=0; tunnels lead to valves AA, JJ",
    "Valve JJ has flow rate=21; tunnel leads to valve II",
];

Hmm, this one seems a bit tricky. The net pressure contribution of a valve is relative to the time we open it, so this is not some fixed shortest path. There is also a chance that backtracking to previously-opened valves might be necessary to reach a new valve.

It looks like the puzzle gives us enough information to calculate:

* At any given time, the net pressure loss each valve can contribute
* The shortest travel times from each valve to every other

Is this a greedy algorithm? No I don't think so - if two valves have equal best pressure release, our subsequent options are then affected by the travel time, so making a locally optimal selection doesn't necessarily yield the globally optimal solution. It seems there is no choice in this case but to evaluate both choices and compare the net result!

Therefore, reluctantly I think we need to compare all possible paths. Normally I would consider this infeasable, but the 30 second timer places an upper bound on the search depth. Further, many valves have `flow rate=0`, which means there's no point opening those! Hopefully those optimisations simplify this enough to make the comparison feasible. Let's see...

In [5]:
// Model the valve first

using ValvePair = (string a, string b);

class Valve(string id, int flowRate)
{
    public string Id { get; init; } = id;
    public int FlowRate { get; init; } = flowRate;
    public List<string> Neighbours { get; } = [];
}

Valve ParseValve(string line)
{
    var id = line[6..8];
    var flowRate = line.ParseInts().Single();

    var valveRegex = Regex.Match(line, "leads? to valves? (.+)$");
    var valveStr = valveRegex.Groups[1].Value;

    Valve v = new(id, flowRate);
    v.Neighbours.AddRange(valveStr.Split(", "));
    return v;
}

In [6]:
// Calculate the shortest paths

using DistanceDict = SCG.Dictionary<(string a, string b), int>;

DistanceDict GetDistances(IEnumerable<Valve> valveList)
{
    var neighbours = valveList.ToDictionary(x => x.Id, x => x.Neighbours);

    NextNodeFunc<string, int> next;
    next = (node, cost) => neighbours[node].Select(n => (n, cost + 1));

    Dictionary<(string, string), int> distances = new();
    var q = from id in neighbours.Keys
            from sp in ShortestPath(id, next)
            select (id, sp.node, sp.cost);

    return q.ToDictionary(qq => (qq.id, qq.node), qq => qq.cost);
}
GetDistances(testInputLines.Select(ParseValve));

In [7]:
// Let's find the answer

int FindMostPressure(string[] inputLines, string startValue)
{
    var valveList = inputLines.Select(ParseValve).ToList();
    var valveDict = valveList.ToDictionary(v => v.Id);
    var distances = GetDistances(valveList);

    const int MaxTime = 30;

    HashSet<string> visited = new();
    int pressure = 0;
    int mostPressure = 0;

    void Dfs(string next, int elapsed)
    {
        if (elapsed >= MaxTime) { return; }
        if (visited.Contains(next)) { return; }

        // At this point we have a valid path!

        // Open valve if it has flow
        elapsed += valveDict[next].FlowRate > 0 ? 1 : 0; 

        var release = valveDict[next].FlowRate * (MaxTime - elapsed);
        pressure += release;
        mostPressure = Math.Max(pressure, mostPressure);

        visited.Add(next);
        foreach (var valve in valveList)
        {
            if (valve is not { Id: var id, FlowRate: > 0}) { continue; }

            var travelTime = distances[(next, id)];
            Dfs(id, elapsed + travelTime);
        }
        visited.Remove(next);
        pressure -= release;
    }

    Dfs(startValue, 0);
    return mostPressure;
}

In [8]:
// This approach lets you release the most pressure possible in 30 minutes with
// this valve layout, 1651.

var testAnswer = FindMostPressure(testInputLines, "AA");
Console.WriteLine(testAnswer);

1651


In [9]:
// Work out the steps to release the most pressure in 30 minutes. What is the
// most pressure you can release?

var part1Answer = FindMostPressure(inputLines, "AA");
Console.WriteLine(part1Answer);

1741


In [10]:
// 1741 is correct!
Ensure(1741, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/16

So in part two, the elephant and I are moving concurrently. How to model this?

The key consequence of having two explorers is that they will be opening valves and thus making them unavailable for opening by the other explorer. So we'll need to model the visited set of valves as a shared state.

In this case, both the elephant and I can move, but concurrent movements will result in some valves being opened by the other party, affecting their partner's search.

I've got two ideas to try here:

1. Model the search as a pair of steps, one for each explorer - This is nice in that the DFS is still a single iteration, but it might be tricky due to travel times meaning the explorers are taking different steps at different times.
2. Execute each search independently - The search in part one gives us every possible combination of path for the person. I think we can use those paths as the "starting point" for the elephant's exploration, and pre-mark the person's valves as visited. I'm pretty sure this gives us every combination?

I think we'll try option 2 first.

In [12]:
int FindMostPressure2(string[] inputLines, string startValue, int maxTime)
{
    var valveList = inputLines.Select(ParseValve).ToList();
    var valveDict = valveList.ToDictionary(v => v.Id);
    var distances = GetDistances(valveList);

    IEnumerable<int> StartDfs(HashSet<string> visited)
    {
        int pressure = 0;

        foreach (var i in Dfs(startValue, 0))
        {
            yield return i;
        }

        IEnumerable<int> Dfs(string next, int elapsed)
        {
            if (elapsed >= maxTime) { yield break; }
            if (visited.Contains(next)) { yield break; }

            // At this point we have a valid path!

            // Open valve if it has flow
            elapsed += valveDict[next].FlowRate > 0 ? 1 : 0; 

            var release = valveDict[next].FlowRate * (maxTime - elapsed);
            pressure += release;
            visited.Add(next);
            yield return pressure;

            foreach (var valve in valveList)
            {
                if (valve is not { Id: var id, FlowRate: > 0}) { continue; }

                var travelTime = distances[(next, id)];
                foreach (var i in Dfs(id, elapsed + travelTime))
                {
                    yield return i;
                }
            }
            visited.Remove(next);
            pressure -= release;
        }
    }

    var best = 0;
    HashSet<string> visited = [];
    foreach (var a in StartDfs(visited))
    {
        visited.Remove(startValue);
        foreach (var b in StartDfs(visited))
        {
            best = Math.Max(best, a + b);
        }
        visited.Add(startValue);
    }
    return best;
}


In [13]:
var part2TestAnswer = FindMostPressure2(testInputLines, "AA", 26);
Console.WriteLine(part2TestAnswer);

1707


In [14]:
// This takes too long to run!

// var part2Answer = FindMostPressure2(inputLines, "AA", 26);
// Console.WriteLine(part2Answer);

Bummer, looks like it works but the execution time is too long! We're on the right track at least, but how can we make it faster? Here's a few ideas...

* No need to reclaculate the DFS each time - the first exploration gives us every possible path and associated pressure release. We can save and compare those paths together: if there is no overlap in paths (besides the initial `AA`), it's a valid pairing.
* Pairing of paths `a, b` is equivalent to `b, a`, so we only need consider _combinations_, not _permutations_ of all pairings.
* Given path `a`, we need only consider the best path `b` that is compatible with `a` - all other paths will yield a lower total. So sorting the paths initially means we can stop once we find our first compatible path `b`, for each `a`.

In [15]:
using PathPressure = (SCG.HashSet<string> path, int pressure);

int FindMostPressure3(string[] inputLines, string startValue, int maxTime)
{
    var valveList = inputLines.Select(ParseValve).ToList();
    var valveDict = valveList.ToDictionary(v => v.Id);
    var distances = GetDistances(valveList);

    HashSet<string> visited = new();
    int pressure = 0;
    List<PathPressure> paths = new();

    void Dfs(string next, int elapsed)
    {
        if (elapsed >= maxTime) { return; }
        if (visited.Contains(next)) { return; }

        // At this point we have a valid path!

        // Open valve if it has flow
        elapsed += valveDict[next].FlowRate > 0 ? 1 : 0; 

        var release = valveDict[next].FlowRate * (maxTime - elapsed);
        pressure += release;
        visited.Add(next);
        paths.Add((new(visited), pressure));
        foreach (var valve in valveList)
        {
            if (valve is not { Id: var id, FlowRate: > 0}) { continue; }

            var travelTime = distances[(next, id)];
            Dfs(id, elapsed + travelTime);
        }
        visited.Remove(next);
        pressure -= release;
    }

    Dfs(startValue, 0);

    // Ok, start the exploration using the optimisations above...

    paths.Sort((a, b) => b.pressure - a.pressure);
    var bestPressure = 0;
    for (var i = 0; i < paths.Count; i++)
    for (var j = i + 1; j < paths.Count; j++)
    {
        PathPressure a = (new(paths[i].path), paths[i].pressure);
        PathPressure b = paths[j];

        a.path.IntersectWith(b.path);
        if (a.path.Count is not 1) { continue; }
        
        var newPressure = a.pressure + b.pressure;
        bestPressure = Math.Max(newPressure, bestPressure);
        break;
    }

    return bestPressure;
}

In [16]:
// With the elephant helping, after 26 minutes, the best you could do would
// release a total of 1707 pressure.

var part2TestAnswer2 = FindMostPressure3(testInputLines, "AA", 26);
Console.WriteLine(part2TestAnswer2);

1707


In [17]:
// With you and an elephant working together for 26 minutes, what is the most
// pressure you could release?

var part2Answer2 = FindMostPressure3(inputLines, "AA", 26);
Console.WriteLine(part2Answer2);

2316


In [18]:
// 2316 is correct!
Ensure(2316, part2Answer2);